In [1]:
!git clone https://github.com/nqh-tq-32052503/Moal.git

Cloning into 'Moal'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 135 (delta 48), reused 41 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 442.67 KiB | 11.96 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
%cd /kaggle/working/Moal/Moal

import os
os.makedirs("/root/autodl-tmp/data/cifar224")

/kaggle/working/Moal/Moal


In [3]:
import json

with open("exps/Moal_cifar224.json", "r") as f:
    args = json.load(f)
print(type(args))

<class 'dict'>


In [4]:
from utils.data_manager import DataManager
from utils import factory
from utils.toolkit import count_parameters

In [5]:
data_manager = DataManager(
        args["dataset"],
        args["shuffle"],
        args["seed"],
        args["init_cls"],
        args["increment"],
        args,
        "/root/autodl-tmp/data/cifar224"
    )

100%|██████████| 169M/169M [00:05<00:00, 32.7MB/s] 


[43, 92, 14, 57, 89, 20, 34, 69, 21, 3, 77, 33, 54, 86, 49, 79, 98, 15, 16, 61, 46, 59, 39, 12, 56, 50, 27, 87, 45, 78, 24, 0, 7, 51, 4, 73, 8, 35, 10, 13, 70, 74, 31, 55, 82, 5, 67, 75, 22, 76, 47, 83, 90, 1, 6, 84, 94, 66, 88, 91, 23, 29, 64, 37, 72, 18, 11, 44, 40, 93, 80, 30, 81, 68, 32, 58, 63, 2, 17, 65, 38, 41, 97, 48, 25, 28, 52, 99, 9, 36, 85, 62, 60, 96, 95, 53, 19, 42, 26, 71]


In [6]:
args["nb_classes"] = data_manager.nb_classes # update args
args["nb_tasks"] = data_manager.nb_tasks
args["model_name"] = "adapt_ac_com_sdc_ema_auto"

In [7]:
model = factory.get_model(args["model_name"], args)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

This is for the BaseNet initialization.
I'm using ViT with adapters.


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

_IncompatibleKeys(missing_keys=['blocks.0.adaptmlp.down_proj.weight', 'blocks.0.adaptmlp.down_proj.bias', 'blocks.0.adaptmlp.up_proj.weight', 'blocks.0.adaptmlp.up_proj.bias', 'blocks.1.adaptmlp.down_proj.weight', 'blocks.1.adaptmlp.down_proj.bias', 'blocks.1.adaptmlp.up_proj.weight', 'blocks.1.adaptmlp.up_proj.bias', 'blocks.2.adaptmlp.down_proj.weight', 'blocks.2.adaptmlp.down_proj.bias', 'blocks.2.adaptmlp.up_proj.weight', 'blocks.2.adaptmlp.up_proj.bias', 'blocks.3.adaptmlp.down_proj.weight', 'blocks.3.adaptmlp.down_proj.bias', 'blocks.3.adaptmlp.up_proj.weight', 'blocks.3.adaptmlp.up_proj.bias', 'blocks.4.adaptmlp.down_proj.weight', 'blocks.4.adaptmlp.down_proj.bias', 'blocks.4.adaptmlp.up_proj.weight', 'blocks.4.adaptmlp.up_proj.bias', 'blocks.5.adaptmlp.down_proj.weight', 'blocks.5.adaptmlp.down_proj.bias', 'blocks.5.adaptmlp.up_proj.weight', 'blocks.5.adaptmlp.up_proj.bias', 'blocks.6.adaptmlp.down_proj.weight', 'blocks.6.adaptmlp.down_proj.bias', 'blocks.6.adaptmlp.up_proj.wei

In [8]:
import logging
cnn_curve, nme_curve = {"top1": [], "top5": []}, {"top1": [], "top5": []}

all_his_acc = []
for task in range(data_manager.nb_tasks):
    logging.info("All params: {}".format(count_parameters(model._network)))
    logging.info(
        "Trainable params: {}".format(count_parameters(model._network, True))
    )
    model.incremental_train(data_manager)
    cnn_accy, nme_accy = model.eval_task()
    model.after_task()

    if nme_accy is not None:
        logging.info("CNN: {}".format(cnn_accy["grouped"]))
        logging.info("NME: {}".format(nme_accy["grouped"]))

        all_his_acc.append(cnn_accy["grouped"].values())
        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        nme_curve["top1"].append(nme_accy["top1"])
        nme_curve["top5"].append(nme_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}".format(cnn_curve["top5"]))
        logging.info("NME top1 curve: {}".format(nme_curve["top1"]))
        logging.info("NME top5 curve: {}\n".format(nme_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        print('Average Accuracy (NME):', sum(nme_curve["top1"])/len(nme_curve["top1"]))

        logging.info("Average Accuracy (CNN): {}".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
        logging.info("Average Accuracy (NME): {}".format(sum(nme_curve["top1"])/len(nme_curve["top1"])))
    else:
        logging.info("No NME accuracy.")
        logging.info("CNN: {}".format(cnn_accy["grouped"]))

        cnn_accy["grouped"].pop('total')
        cnn_accy["grouped"].pop('old')
        cnn_accy["grouped"].pop('new')
        all_his_acc.append(list(cnn_accy["grouped"].values()))

        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}\n".format(cnn_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        logging.info("Average Accuracy (CNN): {} \n".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
logging.info("All History Accuracy (CNN): {} \n".format(all_his_acc))

Learning on 0-10
Train dataset size: 5000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


86,995,969 total parameters.
1,197,313 training parameters.
backbone.blocks.0.adaptmlp.down_proj.weight 49152
backbone.blocks.0.adaptmlp.down_proj.bias 64
backbone.blocks.0.adaptmlp.up_proj.weight 49152
backbone.blocks.0.adaptmlp.up_proj.bias 768
backbone.blocks.1.adaptmlp.down_proj.weight 49152
backbone.blocks.1.adaptmlp.down_proj.bias 64
backbone.blocks.1.adaptmlp.up_proj.weight 49152
backbone.blocks.1.adaptmlp.up_proj.bias 768
backbone.blocks.2.adaptmlp.down_proj.weight 49152
backbone.blocks.2.adaptmlp.down_proj.bias 64
backbone.blocks.2.adaptmlp.up_proj.weight 49152
backbone.blocks.2.adaptmlp.up_proj.bias 768
backbone.blocks.3.adaptmlp.down_proj.weight 49152
backbone.blocks.3.adaptmlp.down_proj.bias 64
backbone.blocks.3.adaptmlp.up_proj.weight 49152
backbone.blocks.3.adaptmlp.up_proj.bias 768
backbone.blocks.4.adaptmlp.down_proj.weight 49152
backbone.blocks.4.adaptmlp.down_proj.bias 64
backbone.blocks.4.adaptmlp.up_proj.weight 49152
backbone.blocks.4.adaptmlp.up_proj.bias 768
backb

100%|██████████| 53/53 [01:00<00:00,  1.15s/it]


Task 0, Epoch 1/2 => Loss 2.200, Train_accy 58.24, Test_accy 96.10


...

Alignment: 100%|██████████| 53/53 [00:34<00:00,  1.56batch/s]

Knowledge Memorization completed.
Updated weight matrix W shape: torch.Size([5000, 100])
Updated correlation matrix R shape: torch.Size([5000, 5000])
numpy inverse
Calibrating classifier weights (Knowledge Rumination - Selective reinforcement of old task knowledge)...



Alignment: 100%|██████████| 521/521 [00:18<00:00, 28.48batch/s]

numpy inverse
Finish one task 


Average Accuracy (CNN): 90.655
